# Implementación del muestreo aleatorio

En este programa implementaremos los elementos necesarios para realizar planificaciones basadas en muestreo. Para ello requeriremos:

1. Crear un conjunto de polígonos que representen los obstáculos en el ambiente.
2. Muestrear puntos en tres dimensiones.
3. Eliminar puntos que caigan en una región de obstáculos poligonal.


In [ ]:
# Importamos paquetes necesarios
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# utilizaremos la librería shapely para representar los polígonos del ambiente
from shapely.geometry import Polygon, Point

%matplotlib inline 
plt.rcParams['figure.figsize'] = 12, 12

In [ ]:
# Los obstáculos están especificados en el siguiente archivo.
filename = 'data_reduced.csv'
data = np.loadtxt(filename, delimiter=',', dtype=np.float64, skiprows=2)

## Crear obstáculos

Para simplificar el proceso, representaremos cada obstáculos mediante una representación de dos partes. La primera será la base de la figura; en este ejemplo supondremos figuras rectangulares. La segunda será la altura. De esta forma, un punto $(x, y, z)$ está en colisión si $(x, y)$ están dentro de la base polygonal y la altura $z$ es menor que la altura del obstáculo.

In [ ]:
def extract_polygons(data):
    # Construye un conjunto de polígonos a partir de los datos de obstáculo
    polygons = []
    for i in range(data.shape[0]):
        north, east, alt, d_north, d_east, d_alt = data[i, :]
        
        # Tarea: Extraer los límites de los obstáculos y colocarlos en una lista
        obstacle = None
        # NOTE: The order of the points matters since
        # `shapely` draws the sequentially from point to point.
        # If the area of the polygon is 0 you've likely got a weird order.
        
        # Determinar las esquinas del polígono como una lista de tuplas
        corners = None 
        
        # Tarea: Determinar la altura del polígono
        height = None

        # Tarea: Definir polígonos a partir de las esquinas
        # Tip: https://shapely.readthedocs.io/en/stable/manual.html#polygons
        #poly = None
        polygons.append((poly, height))

    return polygons

In [ ]:
polygons = extract_polygons(data)

## Muestrear puntos 3D

A continuación generaremos un conjunto de muestras aleatorias dentro de los límites del mapa. 

Lo primero es obtener los límites. Dichos límites los podemos obtener a partir de los valores máximos y mínimos del mapa. Para ello podemos hacer uso de las funciones de *numpy* *min* y *max*. Recordemos también que cada renglón de datos contiene:
[posX, posY, posZ, halfSizeX, halfSizeY, halfSizeZ]. Por lo tanto, debemos combinar la posición con el ancho del objeto.

In [ ]:
xmin = None
xmax = None

ymin = None
ymax = None

zmin = 0
# Limit the z axis for the visualization
zmax = 10

print("X")
print("min = {0}, max = {1}\n".format(xmin, xmax))

print("Y")
print("min = {0}, max = {1}\n".format(ymin, ymax))

print("Z")
print("min = {0}, max = {1}".format(zmin, zmax))

A continuación muestrearemos el espacio usando una función de distribución uniforme. La distribución uniforme nos garantiza que eventualmente muestrearemos todo el espacio. Para ello podemos usar la función de *numpy* *random.uniform*. Las muestras deben de quedar en una lista de tuplas. 

In [ ]:
num_samples = 100

xvals = None
yvals = None
zvals = None

#Tip: zip de tuplas
samples = None

In [ ]:
samples[:5]

## Eliminar puntos que colisionan con los obstáculos

Prior to remove a point we must determine whether it collides with any obstacle. Complete the `collides` function below. It should return `True` if the point collides with *any* obstacle and `False` if no collision is detected.

In [ ]:
def collides(polygons, point):   
    # Tarea: Determinar cuáles puntos están en colisión con los polígonos
    collide = False
    x,y,z = point
    p = Point(x, y)
    # Para todos los polígonos en pyligons
        # Tarea: verifica si un polígono contiene el punto y es menor que la altura, 
        # entonces establece la colisión a True
            
    return collide

Use `collides` for all points in the sample.

In [ ]:
t0 = time.time()
to_keep = []
for point in samples:
    if not collides(polygons, point):
        to_keep.append(point)
time_taken = time.time() - t0
print("Time taken {0} seconds ...", time_taken)

In [ ]:
print(len(to_keep), "puntos válidos.")

## Visualizar los puntos

Ahora solo visualizaremos los puntos.

In [ ]:
from grid import create_grid
grid = create_grid(data, zmax, 1)

In [ ]:
fig = plt.figure()

plt.imshow(grid, cmap='Greys', origin='lower')

nmin = np.min(data[:, 0])
emin = np.min(data[:, 1])

# Draw points
all_pts = np.array(to_keep)
north_vals = all_pts[:,0]
east_vals = all_pts[:,1]
plt.scatter(east_vals - emin, north_vals - nmin, c='green')

plt.ylabel('NORTH')
plt.xlabel('EAST')

plt.show()

## Epilogue

You may have noticed removing points can be quite lengthy. In the implementation provided here we're naively checking to see if the point collides with each polygon when in reality it can only collide with one, the one that's closest to the point. The question then becomes 

"How do we efficiently find the closest polygon to the point?"

One such approach is to use a *[k-d tree](https://en.wikipedia.org/wiki/K-d_tree)*, a space-partitioning data structure which allows search queries in $O(log(n))$. The *k-d tree* achieves this by cutting the search space in half on each step of a query.

This would bring the total algorithm time down to $O(m * log(n))$ from $O(m*n)$.

The scikit-learn library has an efficient implementation [readily available](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree).